In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import contractions
import demoji
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tarang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tarang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_df = pd.read_csv('Resampled Preprocessed Training Data (TaskA).csv')

for i in range(len(train_df.axes[0])):
    words = train_df.iat[i,0][1:-1].split(',')
    
    text = []
    for word in words:
        word = word.strip()[1:-1]
        text.append(word)
        
    train_df.iat[i,0] = text
        
train_df

,text,label_sexist
0,"[wear, name, tag, would, assum]",not sexist
1,"[good, thing, come, back, home, serv, time, be...",not sexist
2,"[stat, countri, mani, want, children, among, c...",not sexist
3,"[feel, like, shit, everyon, hate, win]",not sexist
4,"[say, serious]",not sexist
...,...,...
5771,"[oneiti, bless, curs, smile, enough, brighten,...",sexist
5772,"[mind, game, play, women, involv, one, worth, ...",sexist
5773,"[hate, slug, hate, women]",sexist
5774,"[blah, blah, blah, media, fawn, old, dri, hag,...",sexist


In [3]:
test_df = pd.read_csv('Resampled Preprocessed Testing Data (TaskA).csv')

for i in range(len(test_df.axes[0])):
    words = test_df.iat[i,0][1:-1].split(',')

    text = []
    for word in words:
        word = word.strip()[1:-1]
        text.append(word)

    test_df.iat[i,0] = text
        
test_df

,text,label_sexist
0,"[thank, lord, favor, whoso, findeth, wife, fin...",not sexist
1,"[met, girl, tinder, lost, felt, much, better, ...",not sexist
2,"[lol, cheat, black, woman, pretend, sex, doll,...",not sexist
3,"[guy, grope, ohio, women, sinc, realli, need, ...",not sexist
4,"[sure, laugh, femal, comedian, joke, twice, no...",not sexist
...,...,...
7518,"[sport, women, object, matter]",sexist
7519,"[typic, entitl, femal, bullshit, everyday, peo...",sexist
7520,"[guy, could, care, less, averag, averag, women...",sexist
7521,"[ginger, beer, lot, like, ginger, women, tast,...",sexist


In [4]:
def wordFrequencies(docs):
    all_words = []

    for doc in docs:
        for word in doc:
            all_words.append(word)

    all_words = nltk.FreqDist(all_words)
    return all_words

In [5]:
def kMostFrequentWords(k, all_words):
    words = list(all_words.keys())[:k]
    vocab = set()
    for word in words:
        vocab.add(word)

    vocab = {word:idx for idx, word in enumerate(vocab)}
    return vocab

In [6]:
def prepareTrueFalseFeatureSet(df, feature_words, size):
    feature_vector = []

    for i in range(len(df.axes[0])):
        features = {}
        text = df.iat[i,0]

        for feature_word in feature_words:
            features[feature_word] = feature_word in text

        feature_vector.append(features)
    
    return [(feature_vector[i], df.iat[i,1]) for i in range(len(feature_vector))]

In [7]:
def prepareCountFeatureSet(df, feature_words, size):
    feature_vector = []

    for i in range(len(df.axes[0])):
        features = {}
        text = df.iat[i,0]

        for feature_word in feature_words:
            features[feature_word] = 0
            for word in text:
                if word == feature_word:
                    features[word] += 1

        feature_vector.append(features)
    
    return [(feature_vector[i], df.iat[i,1]) for i in range(len(feature_vector))]

In [8]:
def sklearnKNNClassifier(num_neighbours, train_featuresets, test_featuresets):
    X_train = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in train_featuresets]
    y_train = [feature_tag_tuple[1] for feature_tag_tuple in train_featuresets]
    X_test = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in test_featuresets]
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    knn = KNeighborsClassifier(n_neighbors=num_neighbours)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100, "F1 Macro:", metrics.f1_score(y_test, y_pred, average="macro"))

In [9]:
# for k in [3,7,15]:
#     for size in [500, 1000, 1500, 2000, 3000]:
#         vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#         sklearnKNNClassifier(k, prepareTrueFalseFeatureSet(train_df, vocab, size), prepareTrueFalseFeatureSet(test_df, vocab, size))

#### scikit-learn k-NN Classifier (True/False Features Set)

Number of neighbours = 3, Size of Feature Vector = 500, Accuracy = 82.40, F1 (Macro): 51.83<br>
Number of neighbours = 3, Size of Feature Vector = 1000, Accuracy = 83.34, F1 (Macro): 50.63<br>
Number of neighbours = 3, Size of Feature Vector = 1500, Accuracy = 86.52, F1 (Macro): 51.49<br>
Number of neighbours = 3, Size of Feature Vector = 2000, Accuracy = 85.46, F1 (Macro): 52.34<br>
Number of neighbours = 3, Size of Feature Vector = 3000, Accuracy = 84.69, F1 (Macro): 52.43<br><br>

Number of neighbours = 7, Size of Feature Vector = 500, Accuracy = 57.77, F1 (Macro): 43.10<br>
Number of neighbours = 7, Size of Feature Vector = 1000, Accuracy = 58.82, F1 (Macro): 43.17<br>
Number of neighbours = 7, Size of Feature Vector = 1500, Accuracy = 62.02, F1 (Macro): 44.90<br>
Number of neighbours = 7, Size of Feature Vector = 2000, Accuracy = 62.29, F1 (Macro): 44.66<br>
Number of neighbours = 7, Size of Feature Vector = 3000, Accuracy = 59.55, F1 (Macro): 43.91<br><br>

Number of neighbours = 15, Size of Feature Vector = 500, Accuracy = 62.73, F1 (Macro): 45.72<br>
Number of neighbours = 15, Size of Feature Vector = 1000, Accuracy = 59.46, F1 (Macro): 43.84<br>
Number of neighbours = 15, Size of Feature Vector = 1500, Accuracy = 64.72, F1 (Macro): 46.09<br>
Number of neighbours = 15, Size of Feature Vector = 2000, Accuracy = 66.25, F1 (Macro): 46.24<br>
Number of neighbours = 15, Size of Feature Vector = 3000, Accuracy = 64.20, F1 (Macro): 45.92<br><br>


In [10]:
# for k in [3,7,15]:
#     for size in [500, 1000, 1500, 2000, 3000]:
#         vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#         sklearnKNNClassifier(k, prepareCountFeatureSet(train_df, vocab, size), prepareCountFeatureSet(test_df, vocab, size))

#### scikit-learn k-NN Classifier (Count Features Set)

Number of neighbours = 3, Size of Feature Vector = 500, Accuracy = 81.00, F1 (Macro): 51.23<br>
Number of neighbours = 3, Size of Feature Vector = 1000, Accuracy = 81.56, F1 (Macro): 50.65<br>
Number of neighbours = 3, Size of Feature Vector = 1500, Accuracy = 84.74, F1 (Macro): 51.74<br>
Number of neighbours = 3, Size of Feature Vector = 2000, Accuracy = 81.92, F1 (Macro): 52.07<br>
Number of neighbours = 3, Size of Feature Vector = 3000, Accuracy = 84.08, F1 (Macro): 53.30<br><br>

Number of neighbours = 7, Size of Feature Vector = 500, Accuracy = 58.93, F1 (Macro): 43.43<br>
Number of neighbours = 7, Size of Feature Vector = 1000, Accuracy = 58.67, F1 (Macro): 42.88<br>
Number of neighbours = 7, Size of Feature Vector = 1500, Accuracy = 63.54, F1 (Macro): 45.48<br>
Number of neighbours = 7, Size of Feature Vector = 2000, Accuracy = 62.14, F1 (Macro): 44.92<br>
Number of neighbours = 7, Size of Feature Vector = 3000, Accuracy = 61.61, F1 (Macro): 44.60<br><br>

Number of neighbours = 15, Size of Feature Vector = 500, Accuracy = 62.44, F1 (Macro): 45.25<br>
Number of neighbours = 15, Size of Feature Vector = 1000, Accuracy = 56.75, F1 (Macro): 42.37<br>
Number of neighbours = 15, Size of Feature Vector = 1500, Accuracy = 66.12, F1 (Macro): 46.84<br>
Number of neighbours = 15, Size of Feature Vector = 2000, Accuracy = 67.89, F1 (Macro): 47.43<br>
Number of neighbours = 15, Size of Feature Vector = 3000, Accuracy = 65.53, F1 (Macro): 46.22<br><br>

In [11]:
def sklearnLinearSVC(train_featuresets, test_featuresets):
    X_train = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in train_featuresets]
    y_train = [feature_tag_tuple[1] for feature_tag_tuple in train_featuresets]
    X_test = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in test_featuresets]
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    clf = LinearSVC(fit_intercept=False, loss='hinge', max_iter=5000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred), "F1 Macro:", metrics.f1_score(y_test, y_pred, average="macro"))

In [12]:
# for size in [500, 1000, 1500, 2000, 2500, 3000]:
#     vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#     sklearnLinearSVC(prepareTrueFalseFeatureSet(train_df, vocab, size), prepareTrueFalseFeatureSet(test_df, vocab, size))

#### scikit-learn Linear SVC (True/False Features Set)

With feature vector of size 500, Accuracy: 57.72, F1 (Macro): 43.64<br>
With feature vector of size 1000, Accuracy: 56.84, F1 (Macro): 43.17<br>
With feature vector of size 1500, Accuracy: 59.42, F1 (Macro): 44.79<br>
With feature vector of size 2000, Accuracy: 59.22, F1 (Macro): 44.66<br>
With feature vector of size 2500, Accuracy: 59.87, F1 (Macro): 45.35<br>
With feature vector of size 3000, Accuracy: 59.47, F1 (Macro): 45.07<br>

In [13]:
# for size in [500, 1000, 1500, 2000, 2500, 3000]:
#     vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#     sklearnLinearSVC(prepareCountFeatureSet(train_df, vocab, size), prepareCountFeatureSet(test_df, vocab, size))

#### scikit-learn Linear SVC (Count Features Set)

With feature vector of size 500, Accuracy: 57.05, F1 (Macro): 43.19<br>
With feature vector of size 1000, Accuracy: 57.22, F1 (Macro): 43.33<br>
With feature vector of size 1500, Accuracy: 59.91, F1 (Macro): 45.29<br>
With feature vector of size 2000, Accuracy: 59.39, F1 (Macro): 44.70<br>
With feature vector of size 2500, Accuracy: 60.15, F1 (Macro): 45.43<br>
With feature vector of size 3000, Accuracy: 59.80, F1 (Macro): 45.10<br>

In [14]:
def sklearnDecisionTree(train_featuresets, test_featuresets):
    X_train = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in train_featuresets]
    y_train = [feature_tag_tuple[1] for feature_tag_tuple in train_featuresets]
    X_test = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in test_featuresets]
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    clf = DecisionTreeClassifier(max_depth=11)
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred), "F1 Macro:", metrics.f1_score(y_test, y_pred, average="macro"))

In [15]:
# for size in [500, 1000, 1500, 2000, 3000]:
#     vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#     sklearnDecisionTree(prepareTrueFalseFeatureSet(train_df, vocab, size), prepareTrueFalseFeatureSet(test_df, vocab, size))

#### scikit-learn Decision Tree Classifier (True/False Features Set)

##### max_depth = 11

With feature vector of size  500, Accuracy: 69.33, F1 (Macro): 48.80<br>
With feature vector of size 1000, Accuracy: 69.71, F1 (Macro): 48.82<br>
With feature vector of size 1500, Accuracy: 69.81, F1 (Macro): 49.80<br>
With feature vector of size 2000, Accuracy: 69.67, F1 (Macro): 49.77<br>
With feature vector of size 3000, Accuracy: 70.67, F1 (Macro): 50.55<br>

In [16]:
# for size in [500, 1000, 1500, 2000, 3000]:
#     vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#     sklearnDecisionTree(prepareCountFeatureSet(train_df, vocab, size), prepareCountFeatureSet(test_df, vocab, size))

#### scikit-learn Decision Tree Classifier (Count Features Set)

##### max_depth = 11

With feature vector of size  500, Accuracy: 69.33, F1 (Macro): 48.80<br>
With feature vector of size 1000, Accuracy: 70.12, F1 (Macro): 48.91<br>
With feature vector of size 1500, Accuracy: 69.74, F1 (Macro): 49.70<br>
With feature vector of size 2000, Accuracy: 69.71, F1 (Macro): 49.71<br>
With feature vector of size 3000, Accuracy: 70.42, F1 (Macro): 50.40<br>

In [17]:
def sklearnRandomForest(train_featuresets, test_featuresets):
    X_train = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in train_featuresets]
    y_train = [feature_tag_tuple[1] for feature_tag_tuple in train_featuresets]
    X_test = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in test_featuresets]
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    clf = RandomForestClassifier(n_estimators=100)
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred), "F1 Macro:", metrics.f1_score(y_test, y_pred, average="macro"))

In [18]:
# for size in [500, 1000, 1500, 2000, 3000, 4000, 5000]:
#     vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#     sklearnRandomForest(prepareTrueFalseFeatureSet(train_df, vocab, size), prepareTrueFalseFeatureSet(test_df, vocab, size))

#### scikit-learn Random Forest Classifier (True/False Features Set)

##### num_estimators = 100

With feature vector of size 500, Accuracy: 58.33, F1 (Macro): 44.05<br>
With feature vector of size 1000, Accuracy: 60.61, F1 (Macro): 44.81<br>
With feature vector of size 1500, Accuracy: 62.51, F1 (Macro): 46.33<br>
With feature vector of size 2000, Accuracy: 61.97, F1 (Macro): 46.25<br>
With feature vector of size 3000, Accuracy: 61.45, F1 (Macro): 46.12<br>
With feature vector of size 4000, Accuracy: 65.59, F1 (Macro): 48.91<br>
With feature vector of size 5000, Accuracy: 72.21, F1 (Macro): 53.93<br>

In [19]:
# for size in [500, 1000, 1500, 2000, 3000, 4000, 5000]:
#     vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#     sklearnRandomForest(prepareCountFeatureSet(train_df, vocab, size), prepareCountFeatureSet(test_df, vocab, size))

#### scikit-learn Random Forest Classifier (Count Features Set)

##### num_estimators = 100

With feature vector of size 500, Accuracy: 57.70, F1 (Macro): 43.83<br>
With feature vector of size 1000, Accuracy: 60.23, F1 (Macro): 44.67<br>
With feature vector of size 1500, Accuracy: 61.19, F1 (Macro): 45.75<br>
With feature vector of size 2000, Accuracy: 61.62, F1 (Macro): 46.03<br>
With feature vector of size 3000, Accuracy: 61.03, F1 (Macro): 46.16<br>
With feature vector of size 4000, Accuracy: 63.86, F1 (Macro): 47.89<br>
With feature vector of size 5000, Accuracy: 71.41, F1 (Macro): 53.41<br>

In [20]:
def sklearnMultinomialNB(train_featuresets, test_featuresets):
    X_train = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in train_featuresets]
    y_train = [feature_tag_tuple[1] for feature_tag_tuple in train_featuresets]
    X_test = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in test_featuresets]
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    clf = MultinomialNB()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print(metrics.accuracy_score(y_test, y_pred), metrics.f1_score(y_test, y_pred, average="macro"))

In [21]:
# for size in [500, 1000, 1500, 2000, 2500, 3000]:
#     vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#     sklearnMultinomialNB(prepareTrueFalseFeatureSet(train_df, vocab, size), prepareTrueFalseFeatureSet(test_df, vocab, size))

#### scikit-learn Multinomial Naive Bayes (True/False Features Set)

With feature vector of size  500, Accuracy: 59.22, F1 (Macro): 44.24<br>
With feature vector of size 1000, Accuracy: 56.95, F1 (Macro): 43.23<br>
With feature vector of size 1500, Accuracy: 57.54, F1 (Macro): 44.16<br>
With feature vector of size 2000, Accuracy: 55.87, F1 (Macro): 43.24<br>
With feature vector of size 2500, Accuracy: 55.82, F1 (Macro): 43.46<br>
With feature vector of size 3000, Accuracy: 54.53, F1 (Macro): 42.70<br>

In [22]:
# for size in [500, 1000, 1500, 2000, 2500, 3000]:
#     vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#     sklearnMultinomialNB(prepareCountFeatureSet(train_df, vocab, size), prepareCountFeatureSet(test_df, vocab, size))

#### scikit-learn Multinomial Naive Bayes (Count Features Set)

With feature vector of size  500, Accuracy: 60.16, F1 (Macro): 44.57<br>
With feature vector of size 1000, Accuracy: 58.09, F1 (Macro): 43.79<br>
With feature vector of size 1500, Accuracy: 58.43, F1 (Macro): 44.57<br>
With feature vector of size 2000, Accuracy: 57.07, F1 (Macro): 43.91<br>
With feature vector of size 2500, Accuracy: 56.70, F1 (Macro): 43.91<br>
With feature vector of size 3000, Accuracy: 55.26, F1 (Macro): 43.01<br>

In [23]:
def nltkMultinomialNB(train_featuresets, test_featuresets):
    clf = nltk.NaiveBayesClassifier.train(train_featuresets)
    
    preds = []
    tests = [test_featuresets[i][0] for i in range(len(test_featuresets))]
    for test in tests:
        preds.append(clf.classify(test))
    
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    cnt = 0
    for i in range(len(preds)):
        if preds[i] == y_test[i]:
            cnt += 1
        
    print(cnt/len(preds), metrics.f1_score(y_test, preds, average="macro"))

In [24]:
# for size in [500, 1000, 1500, 2000, 2500, 3000]:
#     vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#     nltkMultinomialNB(prepareTrueFalseFeatureSet(train_df, vocab, size), prepareTrueFalseFeatureSet(test_df, vocab, size))

#### NLTK Multinomial Naive Bayes (True/False Features Set)

With feature vector of size  500, Accuracy: 65.12, F1 (Macro): 47.42<br>
With feature vector of size 1000, Accuracy: 61.97, F1 (Macro): 45.95<br>
With feature vector of size 1500, Accuracy: 62.33, F1 (Macro): 46.79<br>
With feature vector of size 2000, Accuracy: 60.75, F1 (Macro): 45.99<br>
With feature vector of size 2500, Accuracy: 60.00, F1 (Macro): 45.71<br>
With feature vector of size 3000, Accuracy: 58.02, F1 (Macro): 44.46<br>

In [25]:
# for size in [500, 1000, 1500, 2000, 2500, 3000]:
#     vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#     nltkMultinomialNB(prepareCountFeatureSet(train_df, vocab, size), prepareCountFeatureSet(test_df, vocab, size))

#### NLTK Multinomial Naive Bayes (Count Features Set)

With feature vector of size  500, Accuracy: 64.38, F1 (Macro): 46.98<br>
With feature vector of size 1000, Accuracy: 61.61, F1 (Macro): 45.76<br>
With feature vector of size 1500, Accuracy: 61.85, F1 (Macro): 46.29<br>
With feature vector of size 2000, Accuracy: 60.76, F1 (Macro): 45.77<br>
With feature vector of size 2500, Accuracy: 60.14, F1 (Macro): 45.64<br>
With feature vector of size 3000, Accuracy: 58.43, F1 (Macro): 44.75<br>